In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Matplotlib 한글 폰트 설정 시작 ---
from matplotlib import font_manager, rc
import platform

# 운영체제에 따라 폰트 설정
if platform.system() == 'Darwin': # Mac
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
elif platform.system() == 'Linux': # Linux (Colab, Docker 등)
    try:
        import google.colab

        !apt-get update -qq
        !apt-get install fonts-nanum -qq
        font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
        font_name = font_manager.FontProperties(fname=font_path).get_name()
        rc('font', family=font_name)
        font_manager._rebuild() # 폰트 캐시 재빌드
    except ImportError:
        # 일반 Linux 환경에서는 수동 설치 필요 경고
        print("경고: Linux 환경에서 Nanum 폰트가 설치되어 있지 않을 수 있습니다.")
        print("터미널에서 'sudo apt-get install fonts-nanum' 명령으로 설치 후,")
        print("파이썬 환경을 다시 시작해야 한글이 제대로 표시될 수 있습니다.")
        rc('font', family='DejaVu Sans') # 기본 폰트 사용 (한글 깨짐 발생 가능성 있음)
else:
    print("지원되지 않는 운영체제입니다. 한글 폰트 설정이 필요합니다.")

plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호 깨짐 방지
# --- Matplotlib 한글 폰트 설정 끝 ---


# 파일 경로
file_path = 'kobis_boxoffice_latest_202101_202507.csv'

# 1. 데이터 로드
try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"오류: 파일 '{file_path}'를 찾을 수 없습니다. 파일 이름과 경로를 확인해주세요.")
    exit()

# 2. 데이터 클리닝 및 타입 변환
df['audiAcc'] = pd.to_numeric(df['audiAcc'], errors='coerce')
df['salesAcc'] = pd.to_numeric(df['salesAcc'], errors='coerce')
df['targetDt'] = pd.to_datetime(df['targetDt'], errors='coerce', format='%Y%m%d')
df.dropna(subset=['audiAcc', 'salesAcc', 'targetDt'], inplace=True)

# 데이터가 너무 적을 경우 종료
if len(df) < 5:
    print(f"\n경고: 데이터셋의 행 수가 {len(df)}개로 너무 적어 의미 있는 모델 학습을 진행할 수 없습니다.")
    exit()

# 3. 피처 엔지니어링
df['개봉_월'] = df['targetDt'].dt.month

# 4. 독립 변수 (X)와 종속 변수 (y) 분리
# 이 버전에서는 'movieNm'을 X에 포함하지 않습니다. (산점도에 영화명 주석을 달지 않기 때문에)
X = df[['salesAcc', '개봉_월']]
y = df['audiAcc']

# 5. 피처 타입 정의
numerical_features = ['salesAcc']
categorical_features = ['개봉_월']

# 6. 전처리 파이프라인 구축
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# 7. 모델 파이프라인 구축
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', RandomForestRegressor(n_estimators=100,
                                                                  random_state=42,
                                                                  n_jobs=-1))])

# 8. 데이터셋 분할 (학습 세트와 테스트 세트)
test_size_val = max(0.2, 1 / len(X))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_val, random_state=42)

# 9. 모델 학습
model_pipeline.fit(X_train, y_train)

# 10. 예측
y_pred = model_pipeline.predict(X_test)


# --- 산점도 그래프 시각화 (기본 형태) ---
plt.figure(figsize=(10, 6)) # 그래프 크기 설정
sns.scatterplot(x=y_test, y=y_pred, alpha=0.7) # 산점도 그리기: 실제 값(X축) vs 예측 값(Y축)
# alpha는 점의 투명도를 조절하여 겹쳐진 부분을 볼 수 있게 함

# 완벽한 예측을 나타내는 대각선 (y=x) 그리기
# X축의 최소값과 최대값, Y축의 최소값과 최대값을 사용하여 대각선을 그림
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
         'r--', lw=2) # 'r--'는 빨간색 점선을 의미, lw는 선의 두께

# 그래프 제목 및 축 레이블 설정
plt.xlabel("실제 누적 관객수", fontsize=12)
plt.ylabel("예측 누적 관객수", fontsize=12)
plt.title("랜덤 포레스트 모델: 실제 vs 예측 누적 관객수", fontsize=16)
plt.grid(True) # 그리드 표시
plt.show()